# Setup

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import model
import data
import matplotlib.pyplot as plt
import numpy as np
from torchinfo import summary
import torch

## Data Configuration

In [3]:
import zipfile
with zipfile.ZipFile("data.zip", 'r') as zip_ref:
    zip_ref.extractall("data")

In [4]:
height = 512
width = 384
classes = {"cardboard":(0, 403),
        "glass":(1, 501),
        "metal":(2, 410),
        "paper":(3, 594),
        "plastic":(4, 482),
        "trash":(5, 137)}
skin_tones = {0:(41, 23, 9),
              1:(95, 51, 16),
              2:(127, 68, 34),
              3:(178, 102, 68),
              4:(115, 63, 23),
              5:(147, 95, 55),
              6:(173, 138, 96),
              7:(207, 150, 95),
              8:(187, 101, 54),
              9:(212, 158, 122),
              11:(242, 194, 128),
              12:(236, 192, 145),
              13:(249, 212, 160),
              14:(248, 217, 152),
              15:(253, 231, 173),
              16:(254, 227, 197)}

# Model 1

## Data Load (No Mask)

In [5]:
collector = data.DataCollector(height, width, classes)
datasets = collector.collect('data/trashnet', val_size=0.2, test_size=0.2)

Store in X and y


## Construction

In [6]:
unique_filename_model1_unfreeze_1 = "seqno5_model1_train_final_layer"

In [7]:
model1 = model.Model(
    height=height,
    width=width,
    num_classes=len(classes),
    lr=0.001,
    bs = 64,
    rho = 0.9,
    gamma_step = 0.5,
    gamma = 0.9,
    num_active_layers = 5,
    free_all = False,
    if_replace = True,
    workers = 1,
    epochs = 25,
    mask = False,
    unique_filename = unique_filename_model1_unfreeze_1,
    model_type = 1,
    model_weights_ref_path = "ResNet34_Weights.pth"
    )

In [8]:
# summary(model1.model, input_size=(model.batch_size, 3, model.height, model.width))

In [9]:
model1.construct_data(datasets)

## Training (Last Layer)

In [ ]:
_, avg_training_loss, avg_training_accuracy, validation_loss, validation_accuracy = model1.train()

Epoch:   1 	 Training Loss: 1.64663 	 Training Accuracy: 0.32747 	 Validation Loss: 1.48228 	 Validation Accuracy: 0.41782


In [ ]:
test_loss, test_accuracy = model1.test()
test_loss, test_accuracy

## Training (All Layers)

In [ ]:
unique_filename_model1_unfreeze_all = "seqno5_model1_unfreeze_all_layer"
model2 = model.Model(
    height=height,
    width=width,
    num_classes=len(classes),
    lr=0.0001,
    bs = 2,
    rho = 0.9,
    gamma_step = 0.5,
    gamma = 0.9,
    num_active_layers = 5,
    free_all = False,
    if_replace = True,
    workers = 1,
    epochs = 25,
    mask = False,
    unique_filename = unique_filename_model1_unfreeze_all,
    model_type = 2,
    model_weights_ref_path = f'models/{unique_filename_model1_unfreeze_1}/model.pth'
    )

In [ ]:
model2.construct_data(datasets)

In [ ]:
_, avg_training_loss, avg_training_accuracy, validation_loss, validation_accuracy = model2.train()

In [ ]:
test_loss, test_accuracy = model2.test()
test_loss, test_accuracy

# Model2 with mask

## Collect Data (Mask)

In [ ]:
plt.imshow(np.expand_dims(np.array(skin_tones[16]),axis = (0,1)))
plt.show()

In [ ]:
collector_masked = data.DataCollector(height, width, classes, mask_opts = {"mask_size":10,"skintone_label":15})
datasets_masked = collector2.collect('data/trashnet', val_size=0.2, test_size=0.2)

In [ ]:
model3 = model.Model(
    height=height,
    width=width,
    num_classes=len(classes),
    lr=0.001,
    bs = 32,
    rho = 0.9,
    gamma_step = 0.5,
    gamma = 0.9,
    num_active_layers = 5,
    if_replace = True,
    workers = 1,
    epochs = 25,
    mask = True,
    unique_filename = "seqno5_model2",
    model_type = 2,
    model_weights_ref_path = f'models/{unique_filename_model1_unfreeze_all}/model.pth'
    )

In [ ]:
model3.construct_data(datasets_masked)

In [ ]:
_, avg_training_loss, avg_training_accuracy, validation_loss, validation_accuracy = model3.train()

In [ ]:
test_loss, test_accuracy = model3.test()
test_loss, test_accuracy